# Using Heat Templates to Configure Chameleon Instances
In previous tutorials we have gone over how to create and configure instances and [networks](https://jupyter.chameleoncloud.org/import?source=github&src_path=chameleoncloud/notebooks&file_path=examples%2Fconfiguration%2FUsingSSHToConfigureNameResolution.ipynb#Create-Reservations) in a progressively more automated fashion, from using the web interface, to embed instructions in Jupyter Notebooks, and finally using Jupyter Notebook + Ansible. 

This time, we will raise the bar once again and use a heat template to create an entire experiment -- reservation, servers, network, experiment -- without human intervention besides starting the build process.

### Objective

In this notebook you will learn

1. What a heat template is
1. How to write and run one. As before we will create and run an experiment using multiple servers inside a custom network. Yes, it is the same experiment we ran before using iperf.
1. How to set up a reservation in the future that will automatically create your experiment from scratch at the reserved time.

### Limitations
* This tutorial is but an overview of heat templates.
* We will be using the web interface for most of the presentation as it helps us see the steps. However, running Heat Templates is most efficent if running through command line.

### Prerequisites

* You must have a public ssh key already loaded into Chameleon. If you forgot how, you can refer to the instructions on [how to create a SSH keypair](https://jupyter.chameleoncloud.org/import?source=github&src_path=chameleoncloud/notebooks&file_path=tutorials%2Fgetting-started%2FJupyterOrchestration.ipynb#Create-a-SSH-key-pair) from previous webinars.
* You should have read the [intro to Chameleon](https://jupyter.chameleoncloud.org/import?source=github&src_path=chameleoncloud/notebooks&file_path=tutorials%2Fgetting-started%2F#Introduction-to-Chameleon.ipynb)

## Outline
1. [The heat template](#The-heat-template)
1. [Using template](#Using-template)
  - [The experiment](#The-experiment)
  - [GUI](#Using-the-GUI)
  - Command line
1. [Conclusion](#Conclusion)
1. [Further Reading](#Further-Reading)


## The heat template
Heat is a service to build (or orchestrate) and run experiments and applications using yaml-based Heat Orchestration Templates (HOT). The format is very similar to the [AWS CloudFormation](https://docs.aws.amazon.com/AWSCloudFormation/latest/APIReference/Welcome.html) one
### A Simple Template
We will explain how the template works by looking at a simple example. We need to define at least
- Parameters
- Resources
#### Parameters:
That is where we define variables that can be supplied by the user. Some have a default value (in this case the default image is `CC-CentOS7` and the default network is `sharednet1`)
```yaml
  image_name:
    type: string
    label: Image Name
    description: Image to be used by this instance
    default: CC-CentOS7
  network_name:
    type: string
    label: Network Name
    description: Image to be used by this instance
    default: sharednet1
```
Others do not have since is relies on something that is dynamically obtained, such as the reservation ID:
```yaml
  reservation_id:
    type: string
    description: ID of the Blazar reservation to use for launching instances.
    constraints:
    - custom_constraint: blazar.reservation
```
The available heat template versions are listed in https://chi.uc.chameleoncloud.org/project/template_versions/
#### Resources
Resources define what resources are to be deployed and how they should be configured. For instance to define a server we need
  1. The server itself. Note that is relies on parameters that were defined in the parameter session:
```yaml
resources:
  the_server:
    type: OS::Nova::Server
    properties:
      flavor: { get_param: flavor_name }
      image: { get_param: image_name }
      key_name: { get_param: key_name }
      networks:
        - network: { get_param: network_name }
      scheduler_hints: { reservation: { get_param: reservation_id } }
```
  1. The public IP it will be using if this server is the one used to access the rest of the experiments. In this example it is using the public network, which is already defined outside the experiment. So, it relies on nothing in this template.
```yaml
  the_server_floating_ip:
    type: OS::Nova::FloatingIP
    properties:
      pool: public
```
  1. Connect the server to the floating IP. Note that this resource needs the server and its floating IP already defined, so it will wait for them to be created.
```yaml
  the_server_ip_association:
    type: OS::Neutron::FloatingIPAssociation
    # These properties can be used later
    properties:
      floatingip_id: {get_resource: the_server_floating_ip}
      port_id: {get_attr: [the_server, addresses, { get_param: network_name }, 0, port]}
```
If we are creating networks, we also need to define networks, subnets, and routers, and how they interact with the servers. 


## Using template
### The experiment
We have done this experiment before, so we will only take a look at the more interesting steps:
  1. Create the lease/reservation. It should allocate a minimum of 3 servers.
  1. Create network. Most of the code was stolen from the [OpenFlow BYOC Network]( https://www.chameleoncloud.org/appliances/58/) Chameleon appliance.
  1. Create iperf server. Once created it will download, compile, and run iperf (the steps in red):
  ```yaml
  iperf_server:
    type: OS::Nova::Server
    properties:
      flavor: { get_param: flavor_name }
      image: { get_param: image_name }
      key_name: { get_param: key_name }
      networks:
        - network: { get_resource: private_network }
      scheduler_hints: { reservation: { get_param: reservation_id } }
      # This script is run once this server/instance is built
      user_data: |
        #!/bin/bash
        # Install and start iperf
        yum install -y git @development
        git clone https://github.com/esnet/iperf && \
        cd iperf && \
        ./configure && \
        make install && \
        iperf3 -s
  ```
  There is nothing forcing us to run it out as a list of steps here; we could have grabbed a script somewhere and run it here.
  1. Create iperf client. Once created it will iperf against the iperf server, collecting the results into a file.
  1. Create server (ext-server) we will connect to collect the results. There is a script here that is not necessary but shows another example of resource precedence: in hommage to a Chameleon [blog](https://jupyter.chameleoncloud.org/import?source=github&src_path=chameleoncloud/notebooks&file_path=examples%2Fconfiguration%2FUsingSSHToConfigureNameResolution.ipynb), we populate the `/etc/hosts` by obtaining the IP addresses for both the iperf server and client (which is done by the statements inside `params:` below). Since that requires to have these servers up first, that causes the `ext_server` to become available only after the other two servers are up and running.
  ```yaml
    ext_server:
    type: OS::Nova::Server
    properties:
      flavor: { get_param: flavor_name }
      image: { get_param: image_name }
      key_name: { get_param: key_name }
      networks:
        - network: { get_resource: private_network }
      scheduler_hints: { reservation: { get_param: reservation_id } }
      # This script is run once this server/instance is built
      user_data:
        str_replace:
          template: |
            #!/bin/sh
            echo "$iperf_server_ip iperf_server" >> /etc/hosts
            echo "$iperf_client_ip iperf_client" >> /etc/hosts
          params:
            $iperf_server_ip: { get_attr: [ iperf_server, first_address ] }
            $iperf_client_ip: { get_attr: [ iperf_client, first_address ] }
  ```
  Note to be different we are using Bourne shell here instead of Bash. We could also have retrieved a script from some URL and ran it.

### Using the GUI
1. Verify that you have a public key loaded. 
1. Create the lease. We might as well use the web interface, but we also could do it from the [command line](https://jupyter.chameleoncloud.org/import?source=github&src_path=chameleoncloud/notebooks&file_path=examples%2Fconfiguration%2FUsingSSHToConfigureNameResolution.ipynb#Create-Reservations). Remember we need **3** servers.
1. Load template file. 
   1. Go to `Project/Orchestration/Stacks` 
   1. Select `Launch Stack`
   1. Enter the heat template, `iperf.yml`. You can provide a url, cut-n-paste it, or browse for it in your local drive. Then hit `Next`.
   1. Enter the parameters. Next screen should show you the list of those parameters that are defined in the heat template + your password. As we discussed before, some have default values but others, like the lease, do not. The password is needed to run openstack commands as you.
1. Monitoring the progress. If you go to the instances, you will noticed for quite a while the only server listed is `iperf-server`. As `iperf-client` and `ext-server` need information from `iperf-server` (well, `ext-server` needs info from both), `iperf-client` will only be created after `iperf-server` is up and running. And then it will be `ext-server`'s turn; by then the experiment will be concluded since it is a rather fast one. However, we should account for that when we reserve our lease; that will be time we will not have for the experiment.
1. Retrieving the results. There is nothing fancy here, we just have to go through `ext-server` to get the results off `iperf-client`. It should look something like this
   ```bash
   [cc@perftest-ext-server-rus5gg4dbjdi ~]$ cat results                                         
   Connecting to host 192.168.50.101, port 5201                                                         
   [  5] local 192.168.50.82 port 56976 connected to 192.168.50.101 port 5201                           
   [ ID] Interval           Transfer     Bitrate         Retr  Cwnd                                    
   [  5]   0.00-1.00   sec   890 MBytes  7.47 Gbits/sec  394    560 KBytes                               
   [  5]   1.00-2.00   sec  1.10 GBytes  9.42 Gbits/sec    0    663 KBytes                               
   [  5]   2.00-3.00   sec  1.10 GBytes  9.42 Gbits/sec    0    663 KBytes                               
   [  5]   3.00-4.00   sec  1.10 GBytes  9.42 Gbits/sec    0    663 KBytes                               
   [  5]   4.00-5.00   sec  1.09 GBytes  9.41 Gbits/sec    0    672 KBytes                               
   [  5]   5.00-6.00   sec  1.09 GBytes  9.34 Gbits/sec    0    673 KBytes                               
   [  5]   6.00-7.00   sec  1.10 GBytes  9.42 Gbits/sec    0    673 KBytes                               
   [  5]   7.00-8.00   sec   852 MBytes  7.15 Gbits/sec  469    570 KBytes                               
   [  5]   8.00-9.00   sec  1.10 GBytes  9.42 Gbits/sec    0    591 KBytes                               
   [  5]   9.00-10.00  sec  1.10 GBytes  9.42 Gbits/sec    0    609 KBytes                               
   - - - - - - - - - - - - - - - - - - - - - - - - -                                                     
   [ ID] Interval           Transfer     Bitrate         Retr                                          
   [  5]   0.00-10.00  sec  10.5 GBytes  8.99 Gbits/sec  863             sender                        
   [  5]   0.00-10.04  sec  10.5 GBytes  8.95 Gbits/sec                  receiver                                                                                                            
   iperf Done.                                                                                           
   [cc@perftest-ext-server-rus5gg4dbjdi ~]$                                                            
   ```

1. Teardown. Go to `Project/Orchestration/Stacks` and delete it from there. All the resources allocated by this template will be freed.
### Command line
    - Scheduling


## Conclusion
1. Of the different orchestration schemes we have seen so far, Heat Orchestration Templates (HOT) provide the minimum amount of user interaction.
1. HOT can be used in conjunction of Ansible, git, and other technologies.
1. If the experiment requires user intervention, we can use HOT to prep the experiment to the point it is ready for us to use.

## Further Reading
* OpenStack Documentation on [Heat Templates](https://docs.openstack.org/heat/latest/) and their [specs](https://docs.openstack.org/heat/latest/template_guide/hot_spec.html)
* ChameleonCloud [Documentation on using Heat Templates to generate Complex Appliances](https://chameleoncloud.readthedocs.io/en/latest/technical/complex.html). The currently available appliances are listed in its [catalog](https://www.chameleoncloud.org/appliances/).